# 板块-指数-etf
* 板块  
来自券商等自发的把具有相关概念的股票形成组合，称之为板块
* 指数  
来自官方编制的指数
    * 上证系列  
    数量较少
    * 中证系列  
    数量很多
    * 中华系列  
    不清楚
* etf  
等到指数出来后，就会有基金公司成立ETF基金来跟踪购买这些指数

# ETF相关知识

## 国内ETF一共有多少只？其中已经上市的ETF数量是多少？
* 上交所
    * ETF列表：http://www.sse.com.cn/assortment/fund/etf/list/
    * 指数对应的成分股：http://www.sse.com.cn/assortment/fund/list/etfinfo/redemptionlist/index.shtml?FUNDID=510630
* 深交所
    * ETF列表：http://www.szse.cn/market/fund/list/etfList/index.html
    * 指数与样本股：http://www.szse.cn/market/exponent/sample/index.html

# 获取ETF列表
本来想着是直接爬取，结果受挫。。。
* 上交所的爬取异常，直接拷贝的
* 深交所的直接提供下载

## 上交所

In [8]:
# import requests
# from pyquery import PyQuery as pq
# url = 'http://www.sse.com.cn/assortment/fund/etf/list/'
# headers = {
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36 Maxthon/5.2.1.6000',
# }
# url = 'http://query.sse.com.cn/commonQuery.do?jsonCallBack=jsonpCallback12716&isPagination=true&pageHelp.pageSize=25&pageHelp.pageNo=1&pageHelp.beginPage=1&pageHelp.cacheSize=1&pageHelp.endPage=11&sqlId=COMMON_SSE_ZQPZ_ETFLB_L_NEW&type=1'
# r = requests.get(url, headers=headers)
# r.encoding='utf8'
# r.text

In [54]:
import pandas as pd

In [55]:
with open('etflist_shanghai.txt') as f:
    ll = f.readlines()

s = pd.Series(ll)

df = pd.DataFrame(s[s.index%7==0])

df.reset_index(drop=True, inplace=True)

df.columns = ['code']

df.loc[:, 'code'] = df.code.str.strip()

df.loc[:, 'name'] = s[s.index%7==2].tolist()

df.loc[:, 'name'] = df.name.str.strip()

In [56]:
df.loc[:, 'company'] = s[s.index%7==4].tolist()

df.loc[:, 'company'] = df.company.str.strip()

In [57]:
df['source']='sh'

In [58]:
df.head(2)

,code,name,company,source
0,510010,治理ETF,交银施罗德基金管理有限公司,sh
1,510020,超大ETF,博时基金管理有限公司,sh


## 深交所

In [59]:
df_sz = pd.read_excel('ETF_sz.xlsx')

In [60]:
df_sz.head()

,证券代码,证券简称,拟合指数,当前规模(份),基金管理人
0,159994,5GETF,931079,"3,295,001,679",银华基金管理股份有限公司
1,159982,中证500,000905 中证500,"330,211,253",鹏华基金管理有限公司
2,159801,芯片基金,980017,"4,598,560,086",广发基金管理有限公司
3,159979,湾创100,980001,"120,928,947",广发基金管理有限公司
4,159995,芯片ETF,980017,"12,762,463,006",华夏基金管理有限公司


In [61]:
df_sz['source']='sz'

In [62]:
df_sz = df_sz[['证券代码', '证券简称','基金管理人','source']]

In [63]:
df_sz.columns = df.columns.tolist()

In [64]:
df_sz.head(2)

,code,name,company,source
0,159994,5GETF,银华基金管理股份有限公司,sz
1,159982,中证500,鹏华基金管理有限公司,sz


In [65]:
dfc = pd.concat([df, df_sz], ignore_index=True)
dfc.head(2)

,code,name,company,source
0,510010,治理ETF,交银施罗德基金管理有限公司,sh
1,510020,超大ETF,博时基金管理有限公司,sh


In [66]:
dfc.shape

(262, 4)

In [67]:
dfc.loc[:, 'code'] = dfc.source+dfc.code.map(str)

In [68]:
dfc.to_pickle('etflist_sh_sz.pkl')

In [69]:
dfc.head(2)

,code,name,company,source
0,sh510010,治理ETF,交银施罗德基金管理有限公司,sh
1,sh510020,超大ETF,博时基金管理有限公司,sh


# 获取行情

In [70]:
import pandas as pd
from regular_investing_utils import get_hist_k_data

In [152]:
dfc=pd.read_pickle('etflist_sh_sz.pkl')

In [153]:
def ft(s):
    return (s[-1]-s[0])/s[0]

In [154]:
dfc.loc[dfc.code=='sh513100']

,code,name,company,source
158,sh513100,纳指ETF,国泰基金管理有限公司,sh


In [155]:
def cal(code):
    print(code)
    df = get_hist_k_data(code, 1500)
    df.loc[:, 'close'] = df.close.map(float)
    df.loc[:, 'volume'] = df.volume.map(int)
    for time_range in [5+1, 10+1, 20+1, 60+1, 120+1, 250+1, 500+1]:  # 因为使用收盘价计算，所以要加1
        dfc.loc[dfc.code==code, f'{time_range-1}_pct']=df.close.iloc[(-1*time_range):].rolling(time_range).agg(ft).iloc[-1]
    dfc.loc[dfc.code==code, f'close']=df.close.iloc[-1]
    dfc.loc[dfc.code==code, f'vol_mean_20']=int(df.volume.iloc[-20:].mean()) # 近20个交易日平均成交量
    dfc.loc[dfc.code==code, f'amount_mean_20']=int(df.close.iloc[-20:].mean()*dfc.loc[dfc.code==code, f'vol_mean_20']) # 近20个交易日平均成交额
    
    df.to_pickle(f'fund_daily_data/{code}_1500.pkl')

In [156]:
import time

In [157]:
num = dfc.shape[0]
cnt = 0

while cnt < num:
    try:
        cal(dfc.code.iloc[cnt])
        cnt += 1
#         break
    except:
        time.sleep(90)

sh510010
sh510020
sh510030
sh510050
sh510060
sh510070
sh510090
sh510100
sh510110
sh510120
sh510130
sh510150
sh510160
sh510170
sh510180
sh510190
sh510210
sh510220
sh510230
sh510270
sh510290
sh510410
sh510430
sh510440
sh510600
sh510630
sh510650
sh510660
sh510680
sh510710
sh510800
sh510810
sh510850
sh510880
sh510890
sh510300
sh510310
sh510330
sh510350
sh510360
sh510380
sh510390
sh510500
sh510510
sh510530
sh510550
sh510560
sh510580
sh510590
sh512000
sh512010
sh512040
sh512070
sh512090
sh512100
sh512120
sh512150
sh512160
sh512170
sh512180
sh512190
sh512200
sh512220
sh512260
sh512270
sh512280
sh512290
sh512300
sh512310
sh512330
sh512340
sh512360
sh512380
sh512390
sh512400
sh512480
sh512500
sh512510
sh512520
sh512530
sh512550
sh512560
sh512570
sh512580
sh512590
sh512600
sh512610
sh512640
sh512650
sh512660
sh512670
sh512680
sh512690
sh512700
sh512710
sh512720
sh512730
sh512750
sh512760
sh512770
sh512780
sh512790
sh512800
sh512810
sh512820
sh512850
sh512860
sh512870
sh512880
sh512890
sh512900
s

In [158]:
dfc.to_pickle('etflist_sh_sz_190307.pkl')

In [159]:
dfc.tail()

,code,name,company,source,5_pct,10_pct,20_pct,60_pct,120_pct,250_pct,500_pct,close,vol_mean_20,amount_mean_20
257,sz159906,深成长,大成基金管理有限公司,sz,0.046414,-0.007206,0.139706,0.388578,0.438515,0.538462,0.215686,1.240,1409957.0,1694415.0
258,sz159905,深红利,工银瑞信基金管理有限公司,sz,0.059039,0.002473,0.075902,0.055180,0.051349,0.238998,0.175754,2.027,31589002.0,62509896.0
259,sz159903,深成ETF,南方基金管理股份有限公司,sz,0.053068,-0.004702,0.090129,0.192488,0.201514,0.301230,0.134942,1.270,2378723.0,2956871.0
260,sz159902,中 小 板,华夏基金管理有限公司,sz,0.045442,-0.017246,0.080829,0.232202,0.237141,0.272852,0.095210,3.704,16830696.0,61481690.0
261,sz159901,深100ETF,易方达基金管理有限公司,sz,0.058934,0.000000,0.092141,0.161622,0.189542,0.319766,0.179101,5.642,29564809.0,162387669.0


In [160]:
dfc.shape

(262, 14)

In [161]:
dfc.dtypes

code               object
name               object
company            object
source             object
5_pct             float64
10_pct            float64
20_pct            float64
60_pct            float64
120_pct           float64
250_pct           float64
500_pct           float64
close             float64
vol_mean_20       float64
amount_mean_20    float64
dtype: object

In [162]:
dfc.sort_values('5_pct', ascending=False).head(20)

,code,name,company,source,5_pct,10_pct,20_pct,60_pct,120_pct,250_pct,500_pct,close,vol_mean_20,amount_mean_20
100,sh512780,京津冀基,广发基金管理有限公司,sh,0.112549,0.084489,0.197772,0.184573,0.155914,0.056511,NaN,0.860,4.463247e+06,3.525072e+06
237,sz159928,消费ETF,汇添富基金管理股份有限公司,sz,0.087653,0.034382,0.101554,0.082117,0.036399,0.436486,0.433258,3.189,3.333153e+07,1.011495e+08
110,sh512900,证券基金,南方基金管理股份有限公司,sh,0.084200,0.001921,0.152486,0.181200,0.133696,0.026575,0.144896,1.043,3.996736e+07,3.975754e+07
120,sh515010,华夏证券,华夏基金管理有限公司,sh,0.084066,0.006352,0.155208,0.192473,NaN,NaN,NaN,1.109,9.462150e+06,9.990611e+06
82,sh512570,中证证券,易方达基金管理有限公司,sh,0.083686,0.013875,0.158550,0.181293,0.135405,0.023000,0.055728,1.023,2.001539e+06,1.941492e+06
49,sh512000,券商ETF,华宝基金管理有限公司,sh,0.083156,0.001972,0.144144,0.177289,0.127636,0.018036,0.127636,1.016,1.087546e+09,1.056769e+09
50,sh512010,医药ETF,易方达基金管理有限公司,sh,0.081933,0.028971,0.037261,0.124454,0.159257,0.335062,0.289111,2.060,5.775054e+07,1.149553e+08
108,sh512880,证券ETF,国泰基金管理有限公司,sh,0.081054,0.002820,0.148547,0.176406,0.129101,0.022031,0.121977,1.067,2.145894e+09,2.190529e+09
184,sz159993,龙头券商,鹏华基金管理有限公司,sz,0.080164,-0.000951,0.132543,NaN,NaN,NaN,NaN,1.051,1.092333e+08,1.101018e+08
58,sh512170,医疗ETF,华宝基金管理有限公司,sh,0.079652,0.066094,0.121697,0.310317,0.369270,NaN,NaN,1.613,6.931814e+07,1.050482e+08


In [163]:
dfc.sort_values('5_pct', ascending=True).head(20)

,code,name,company,source,5_pct,10_pct,20_pct,60_pct,120_pct,250_pct,500_pct,close,vol_mean_20,amount_mean_20
156,sh513030,德国30,华安基金管理有限公司,sh,-0.049856,-0.108813,-0.109614,-0.097450,-0.021718,0.014330,-0.095803,0.991,5.736040e+06,6.219014e+06
181,sz159801,芯片基金,广发基金管理有限公司,sz,-0.034790,-0.130924,NaN,NaN,NaN,NaN,NaN,1.082,8.005941e+08,9.472744e+08
75,sh512480,半导体,国联安基金管理有限公司,sh,-0.032432,-0.138293,0.067787,0.499619,0.631317,NaN,NaN,1.969,2.626766e+08,5.448437e+08
183,sz159995,芯片ETF,华夏基金管理有限公司,sz,-0.031414,-0.130094,NaN,NaN,NaN,NaN,NaN,1.110,1.931516e+09,2.261805e+09
160,sh513520,日经ETF,华夏基金管理有限公司,sh,-0.029211,-0.069094,-0.101802,-0.096920,-0.010913,NaN,NaN,0.997,9.929885e+06,1.053561e+07
98,sh512760,半导体50,国泰基金管理有限公司,sh,-0.026777,-0.134292,0.060896,0.491861,0.625289,NaN,NaN,2.108,9.782068e+08,2.171961e+09
165,sh513880,日经225,华安基金管理有限公司,sh,-0.024802,-0.069129,-0.097337,-0.089815,-0.017000,NaN,NaN,0.983,3.416920e+06,3.570681e+06
241,sz159926,国债ETF,嘉实基金管理有限公司,sz,-0.022881,0.021339,0.045132,0.041074,0.019903,0.049613,0.057798,115.300,6.019000e+03,6.770570e+05
221,sz159941,纳指ETF,广发基金管理有限公司,sz,-0.017641,-0.108417,-0.101015,0.040577,0.114989,0.264763,0.457741,1.949,1.650847e+07,3.549156e+07
155,sh513000,225ETF,易方达基金管理有限公司,sh,-0.014735,-0.073013,-0.100448,-0.089837,-0.009872,NaN,NaN,1.003,3.821415e+06,4.076303e+06


In [131]:
dfc[dfc.name.isin(target_pool)]

,code,name,company,source,5_pct,10_pct,20_pct,60_pct,120_pct,250_pct,500_pct,close,vol_mean_20,amount_mean_20
35,sh510300,300ETF,华泰柏瑞基金管理有限公司,sh,0.011040,0.002189,0.056396,0.045408,0.041446,0.122888,0.076823,4.121,455758322.0,1.846345e+09
42,sh510500,500ETF,南方基金管理股份有限公司,sh,0.017151,-0.017358,0.069564,0.159806,0.128284,0.161104,0.066998,6.227,283185069.0,1.727217e+09
243,sz159922,500ETF,嘉实基金管理有限公司,sz,0.021077,-0.013575,0.073136,0.165108,0.139869,0.168453,0.072382,6.104,30864444.0,1.839212e+08
246,sz159919,300ETF,嘉实基金管理有限公司,sz,0.018204,0.003348,0.058809,0.065803,0.064721,0.143675,-0.041143,4.195,181916614.0,7.485232e+08
248,sz159915,创业板,易方达基金管理有限公司,sz,0.027751,-0.030762,0.076492,0.280947,0.313628,0.447874,0.405459,2.111,569149929.0,1.178169e+09


# 求相关系数

In [1]:
import pandas as pd

In [119]:
num = dfc.shape[0]
cnt = 0
dfa = pd.DataFrame()
while cnt < num:
    dft = pd.read_pickle(f'fund_daily_data/{dfc.code.iloc[cnt]}_1500.pkl')
    dfa[dfc.name.iloc[cnt]] = dft.close/dft.close.shift(1) - 1  # 简单收益率
    cnt += 1
    

In [132]:
dft = pd.read_pickle(f'fund_daily_data/sh510300_1500.pkl')

In [133]:
dft.tail(10)

,close,day,high,low,open,volume
day,,,,,,
2020-02-24,4.112,2020-02-24,4.136,4.082,4.123,372727289
2020-02-25,4.104,2020-02-25,4.116,4.012,4.051,603830762
2020-02-26,4.070,2020-02-26,4.118,4.048,4.064,641793391
2020-02-27,4.071,2020-02-27,4.097,4.061,4.083,475330532
2020-02-28,3.933,2020-02-28,4.019,3.924,4.01,560071042
2020-03-02,4.076,2020-03-02,4.09,3.948,3.948,426683708
2020-03-03,4.073,2020-03-03,4.139,4.055,4.117,408776061
2020-03-04,4.102,2020-03-04,4.104,4.048,4.065,295453297
2020-03-05,4.200,2020-03-05,4.208,4.111,4.151,547831292


In [126]:
target_pool = '300ETF 500ETF 创业板'.split()

In [129]:
dfa_corr = dfa[target_pool].iloc[-60:].corr()

In [130]:
dfa_corr

,300ETF,500ETF,创业板
300ETF,1.000000,0.912582,0.847824
500ETF,0.912582,1.000000,0.924411
创业板,0.847824,0.924411,1.000000


In [121]:
dfa_corr['300ETF'].sort_values(ascending=False)

300ETF      1.000000
民生300       0.991118
HS300       0.984706
天弘300       0.984308
工银300       0.983275
银华MSCI      0.973428
800ETF      0.971920
添富300       0.971598
深100SZ      0.970028
深100ET      0.969098
长三角         0.968432
SZ50ETF     0.968038
ZZ800ETF    0.965049
工银上50       0.962945
深100P       0.959218
小盘价值        0.956927
HS300ETF    0.953243
制造100       0.952648
中证红利        0.950737
100红利       0.950164
湾区基金        0.950032
500ETFEW    0.946492
南方300       0.942780
龙头券商        0.941163
景顺MSCI      0.940840
华夏300       0.940346
MSCI易基      0.940163
湾区ETF       0.939585
300低波       0.936481
MSCI基金      0.935547
              ...   
225ETF      0.461662
港股100       0.460570
非周ETF       0.454406
日经ETF       0.421653
日经225       0.417506
建信H股        0.415871
芯片基金        0.389793
东证ETF       0.377344
纳指ETF       0.359604
添富国企        0.354486
德国30        0.349126
芯片ETF       0.341419
标普500       0.233852
深100ETF     0.159101
中期信用        0.112082
国债十年        0.028383
保证金         0

In [128]:
dfa_corr[target_pool]

,300ETF,500ETF,创业板
治理ETF,0.883101,0.824162,0.700848
超大ETF,0.935706,0.858070,0.789740
价值ETF,0.939486,0.810605,0.722254
50ETF,0.983141,0.848282,0.778285
央企ETF,0.941239,0.927070,0.805885
民企ETF,0.809235,0.806962,0.819643
责任ETF,0.458216,0.475104,0.389144
SZ50ETF,0.972210,0.841824,0.786638
周期ETF,0.699322,0.646236,0.567015
非周ETF,0.612896,0.646694,0.568503


In [112]:
dfa

,sh510010,sh510020,sh510030,sh510050,sh510060,sh510070,sh510090,sh510100,sh510110,sh510120,...,sz159911,sz159910,sz159909,sz159907,sz159908,sz159906,sz159905,sz159903,sz159902,sz159901
day,,,,,,,,,,,,,,,,,,,,,
2014-01-02,NaN,-0.005886,-0.011354,NaN,NaN,0.008897,0.011378,NaN,0.000000,-0.018722,...,0.012241,-0.005298,0.010292,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2014-01-03,-0.003205,-0.013816,-0.016784,NaN,NaN,-0.003527,-0.016250,NaN,-0.021891,-0.003367,...,0.004031,0.002663,0.007709,NaN,-0.011445,-0.006623,NaN,NaN,NaN,NaN
2014-01-06,-0.020900,-0.020680,-0.017969,NaN,NaN,-0.028319,0.002541,NaN,-0.009156,-0.036599,...,-0.024707,-0.029216,-0.016393,NaN,0.005789,-0.014667,NaN,NaN,NaN,NaN
2014-01-07,-0.001642,0.000681,-0.002287,NaN,NaN,0.007286,-0.002535,NaN,0.003593,0.009351,...,0.003800,0.002736,-0.003611,NaN,-0.010072,NaN,NaN,NaN,NaN,NaN
2014-01-08,0.000000,-0.004765,0.000000,NaN,NaN,0.004521,-0.013977,NaN,-0.002558,0.013318,...,0.013880,0.006821,0.011709,NaN,-0.010174,NaN,NaN,NaN,NaN,NaN
2014-01-09,-0.001645,-0.004788,0.000459,NaN,NaN,-0.000900,-0.032216,NaN,-0.005128,0.002857,...,-0.002800,-0.037940,-0.007165,NaN,-0.008811,-0.005413,NaN,NaN,NaN,-0.010989
2014-01-10,-0.006590,-0.004811,-0.003208,-0.003992,-0.006257,-0.022523,0.025300,NaN,0.011856,-0.028490,...,-0.011856,0.001408,-0.006384,-0.022199,-0.001481,-0.005442,-0.004601,-0.011194,-0.018876,-0.012963
2014-01-13,-0.013267,0.000691,0.000000,-0.004008,0.001049,-0.016590,NaN,NaN,-0.021396,NaN,...,-0.018945,0.054852,-0.008380,-0.005405,0.000000,0.000000,-0.021572,-0.011321,-0.004810,-0.009381
2014-01-14,0.010084,-0.000690,0.004138,0.002683,0.002096,0.001874,0.000000,NaN,0.011973,-0.017009,...,0.007403,-0.005333,0.022535,0.016304,0.001484,0.002736,0.018898,0.011450,0.014499,0.013258


In [108]:
dfa = pd.DataFrame()

In [106]:
df = pd.read_pickle(f'fund_daily_data/sh510330_1500.pkl')

In [107]:
df.head()

,close,day,high,low,open,volume
day,,,,,,
2014-01-09,2.261,2014-01-09,2.295,2.258,2.272,135359600
2014-01-10,2.243,2014-01-10,2.257,2.238,2.255,136825248
2014-01-13,2.227,2014-01-13,2.256,2.219,2.245,247165456
2014-01-14,2.243,2014-01-14,2.248,2.214,2.233,36169100
2014-01-15,2.242,2014-01-15,2.246,2.224,2.245,29694000


In [109]:
dfa['sh510330'] = df.close

In [110]:
dfa

,sh510330
day,
2014-01-09,2.261
2014-01-10,2.243
2014-01-13,2.227
2014-01-14,2.243
2014-01-15,2.242
2014-01-16,2.244
2014-01-17,2.213
2014-01-20,2.202
2014-01-21,2.217


In [50]:
df2 = pd.read_pickle(f'fund_daily_data/sz159937_190219_1500.pkl')

In [51]:
df2.head()

,close,day,high,low,open,volume
day,,,,,,
2015-01-08,2.43,2015-01-08,2.436,2.41,2.412,13784
2015-01-09,2.421,2015-01-09,2.421,2.41,2.417,28192
2015-01-12,2.428,2015-01-12,2.45,2.421,2.45,9000
2015-01-13,2.474,2015-01-13,2.475,2.437,2.437,6468
2015-01-14,2.44,2015-01-14,2.495,2.439,2.439,12476


In [46]:
s1 = (df.close/df.close.shift(1)-1).map(float)

In [52]:
s2 = (df2.close/df2.close.shift(1)-1).map(float)

In [53]:
s1.corr(s2)

-0.030408305245788917